In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv(r"/content/drive/My Drive/Colab Notebooks/ppr/D2-Conrad/S1/Office_Conrad-S1.csv",index_col='timestamp')

In [4]:
train_data = df.iloc[:1440]
test_data = df.iloc[1440:]

In [5]:
len(train_data),len(test_data)

(1440, 720)

In [6]:
def load_data(data, seq_len):
    X = []
    y = []
    #print(len(data))
    for i in range(seq_len, len(data),24):
        X.append(data.iloc[i-seq_len : i, 0])
        y.append(data.iloc[i:i+24, 0])
        #print(y)
   
    
    return [X, y]

In [7]:
# Sequence of training data
seq_len = 24 #choose sequence length

X1, y1= load_data(train_data, seq_len)

In [8]:
#Sequence of testing data
seq_len = 24 #choose sequence length

X2, y2= load_data(test_data, seq_len)

In [9]:
X_train = X1      
y_train = y1
X_test = X2             
y_test = y2

In [10]:
# convert training and testing data to numpy array
X_train = np.array(X_train)
y_train = np.array(y_train)
  
X_test = np.array(X_test)
y_test = np.array(y_test)

# reshape data to input into 3 dimension
X_train = np.reshape(X_train, (X_train.shape[0], seq_len, 1))
X_test = np.reshape(X_test, (X_test.shape[0], seq_len, 1))

In [11]:
print('X_train.shape = ',X_train.shape)
print('y_train.shape = ',y_train.shape)
print('X_test.shape = ',X_test.shape)
print('y_test.shape = ',y_test.shape)

X_train.shape =  (59, 24, 1)
y_train.shape =  (59, 24)
X_test.shape =  (29, 24, 1)
y_test.shape =  (29, 24)


In [12]:
import tensorflow as tf
import sklearn.preprocessing
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.models import Sequential

In [13]:
# define model
model = Sequential()
model.add(LSTM(24, activation='tanh', return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(24, activation='tanh'))
model.add(Dense(24))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 24, 24)            2496      
                                                                 
 lstm_1 (LSTM)               (None, 24)                4704      
                                                                 
 dense (Dense)               (None, 24)                600       
                                                                 
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(optimizer="adam",loss="MSE")
model.fit(X_train, y_train, epochs=300, batch_size=128)

Epoch 1/300
1/1 [==============================] - 4s 4s/step - loss: 130.0476
Epoch 2/300
1/1 [==============================] - 0s 27ms/step - loss: 129.7135
Epoch 3/300
1/1 [==============================] - 0s 29ms/step - loss: 129.3876
Epoch 4/300
1/1 [==============================] - 0s 31ms/step - loss: 129.0639
Epoch 5/300
1/1 [==============================] - 0s 26ms/step - loss: 128.7379
Epoch 6/300
1/1 [==============================] - 0s 29ms/step - loss: 128.4057
Epoch 7/300
1/1 [==============================] - 0s 26ms/step - loss: 128.0641
Epoch 8/300
1/1 [==============================] - 0s 37ms/step - loss: 127.7102
Epoch 9/300
1/1 [==============================] - 0s 26ms/step - loss: 127.3413
Epoch 10/300
1/1 [==============================] - 0s 27ms/step - loss: 126.9547
Epoch 11/300
1/1 [==============================] - 0s 32ms/step - loss: 126.5477
Epoch 12/300
1/1 [==============================] - 0s 28ms/step - loss: 126.1181
Epoch 13/300
1/1 [=========

In [15]:
train_prediction=model.predict(X_train)
test_prediction=model.predict(X_test)

In [16]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

from math import sqrt

In [17]:
rmse_train = sqrt(mean_squared_error(y_train, train_prediction))
rmse_test = sqrt(mean_squared_error(y_test, test_prediction))

mae_train=mean_absolute_error(y_train, train_prediction)
mae_test=mean_absolute_error(y_test, test_prediction)

mape_train = mean_absolute_percentage_error(y_train, train_prediction)
mape_test = mean_absolute_percentage_error(y_test, test_prediction)

In [18]:
print('Root Mean Squared Error for Stacked LSTM Model for training is:',rmse_train)
print('Root Mean Squared Error for Stacked LSTM Model for testing is:',rmse_test)
print('Mean Absolute Error for Stacked LSTM Model for training is:',mae_train)
print('Mean Absolute Error for Stacked LSTM Model for testing is:',mae_test)
print('MAPE for Stacked Model for training is:',mape_train)
print('MAPE for Stacked Model for testing is:',mape_test)

Root Mean Squared Error for Stacked LSTM Model for training is: 4.285223213671577
Root Mean Squared Error for Stacked LSTM Model for testing is: 3.6585377719103023
Mean Absolute Error for Stacked LSTM Model for training is: 3.6364127076878883
Mean Absolute Error for Stacked LSTM Model for testing is: 3.0363916043014925
MAPE for Stacked Model for training is: 0.30124684937804663
MAPE for Stacked Model for testing is: 0.2599893808643026


In [ ]:
#pred=model.predict(X_test)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,8)
plt.plot(test_prediction,label='Stacked LSTM_Predictions')
plt.plot(y_test,label='Actual value')
plt.legend(loc="upper left")
plt.show()

In [ ]:
y_test=y_test.reshape(-1,1)

y_test[0],test_prediction[0]

(array([2.5999999]), array([2.8415837], dtype=float32))

In [ ]:
d1=[]
for i in range(len(y_test)):
    d1.append(y_test[i]-test_prediction[i])
    
print(d1)

[array([-0.24158382]), array([0.02406812]), array([0.30863547]), array([0.06328011]), array([-0.57702422]), array([-0.00120115]), array([-0.08095694]), array([-0.18164158]), array([-0.50208902]), array([0.35607338]), array([-0.26854038]), array([0.07756233]), array([0.19879484]), array([0.97497511]), array([0.72668552]), array([-0.00062036]), array([0.038239]), array([-0.04140091]), array([-0.31018639]), array([-0.54930973]), array([0.08351922]), array([-0.90226889]), array([-0.1347487]), array([-0.67985511]), array([-0.01477814]), array([-1.11499083]), array([-0.12798262]), array([-0.20550442]), array([-0.28494024]), array([0.66055703]), array([-0.49606133]), array([0.28300381]), array([-0.57748413]), array([-0.3552599]), array([0.43375349]), array([-0.48887634]), array([0.22115517]), array([0.12760305]), array([0.84573221]), array([-0.40832281]), array([0.33250046]), array([0.59936333]), array([-0.13603544]), array([-0.29132128]), array([1.10199904]), array([-0.21402741]), array([-0.

In [ ]:
d1.sort()
# Range of error value
max(d1),min(d1)

(array([1.74992371]), array([-2.16093707]))

In [ ]:
neg=[]
pos=[]
for i in range(len(d1)):
  if(d1[i]<0):
    neg.append(d1[i])
  if(d1[i]>0):  
    pos.append(d1[i])

In [ ]:
len(neg),len(pos)

(458, 238)